In [1]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# Connect to Redshift
conn = psycopg2.connect(
    dbname=os.getenv('REDSHIFT_SCHEMA'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    host=os.getenv('REDSHIFT_HOST'),
    port=5439
)

cur = conn.cursor()

# Example schema — adjust columns to match your CSV
create_table_sql = """
DROP TABLE IF EXISTS DAILY_CRIPTO_PRICES;

CREATE TABLE DAILY_CRIPTO_PRICES (
    usd FLOAT,
    cripto VARCHAR(100),
    date TIMESTAMP
);
"""

cur.execute(create_table_sql)
conn.commit()

cur.close()

In [8]:
import pandas as pd

df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)
print(df.head())

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_7206/2122793251.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)


Empty DataFrame
Columns: [usd, cripto, date]
Index: []


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Read local CSV
df = pd.read_csv("mydata.csv")

# Create SQLAlchemy engine
engine = create_engine(
    "postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}".format(
        user="your_user",
        password="your_password",
        host="your-cluster.redshift.amazonaws.com",
        port=5439,
        dbname="your_db"
    )
)

# Insert into existing table
df.to_sql("my_table", engine, index=False, if_exists="append")